![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [145]:
import pandas as pd
import numpy as np

# Start coding here...

In [146]:
df = pd.read_csv("bank_marketing.csv")

client = df[["client_id", "age", "job", "marital","education", "credit_default", "mortgage"]]

client["job"]=client["job"].replace(".","_")
client["education"]=client["education"].str.replace(".","_")
client["education"]=client["education"].replace("unknown", np.NAN)

client["credit_default"]=client["credit_default"].apply(lambda x: 1 if x=="yes" else 0)
client["credit_default"] = client['credit_default'].astype(bool)

client["mortgage"]=client["mortgage"].apply(lambda x: 1 if x=="yes" else 0)
client['mortgage'] = client['mortgage'].astype(bool)

client.to_csv("client.csv",index=False)


In [147]:

if 'last_contact_date' not in df.columns:
    df['month'] = pd.to_datetime(df['month'], format='%b').dt.month
    df['last_contact_date'] = pd.to_datetime('2022' + '-' + df['month'].astype(str) + '-' + df['day'].astype(str), format="%Y-%m-%d")

campaign = df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts","previous_outcome","campaign_outcome", "last_contact_date"]]

campaign["previous_outcome"] = campaign["previous_outcome"].apply(lambda x: 1 if x=="success" else 0)
campaign["previous_outcome"] = campaign["previous_outcome"].astype(bool)
campaign["campaign_outcome"] = campaign["campaign_outcome"].apply(lambda x: 1 if x=="yes" else 0)
campaign["campaign_outcome"] = campaign["campaign_outcome"].astype(bool)

campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"])

campaign.to_csv("campaign.csv", index=False)

In [148]:

economics = df[["client_id", "cons_price_idx", "euribor_three_months"]]

economics.to_csv("economics.csv",index=False)